In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pytorch_lightning
from nlp_datasets import get_dataloader, get_dataset

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter
from accelerate import Accelerator
from safetensors.torch import load_file

import json
import gc

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
writer = SummaryWriter()
accelerator = Accelerator()

In [3]:
torch.cuda.empty_cache()
gc.collect()

60

In [4]:
text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    text_decoder_id,
    lagacy = False
)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
for name, module in base_model.named_modules():
    print(name)
    break

In [7]:
train_dataloader = get_dataloader()
valid_dataloader = get_dataloader(split='valid')

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[자본시장과금융투자업에관한법률위반]〈이른바 스캘핑(scalping) 행위가 자본시장법위반죄로 기소된 사건〉 [공2017상,930]\n판시사항:투자자문업자 등이, 추천하는 증권을 자신이 선행매수하여 보유하고 있고 추천 후에 이를 매도할 수도 있다는 그 증권에 관한 자신의 이해관계를 표시하지 않은 채 그 증권의 매수를 추천하는 행위가 자본시장과 금융투자업에 관한 법률 제178조 제1항 제1호에서 말하는 ‘부정한 수단, 계획, 기교를 사용하는 행위’ 및 같은 조 제2항에서 정한 ‘위계의 사용’에 해당하는지 여부(적극)\n\n        assistant : \n    ', 'label': '\n그러므로 상고를 기각하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 판결한다.\n상고이유를 판단한다.\n1. 자본시장과 금융투자업에 관한 법률 제178조 제1항 제1호, 제2항에 관한 법리오해 등의 점에 대하여\n가. 1) 자본시장과 금융투자업에 관한 법률(이하 ‘자본시장법’이라고 한다) 제178조 제1항 제1호는 금융투자상품의 매매, 그 밖의 거래와 관련하여 ‘부정한 수단, 계획 또는 기교를 사용하는 행위’를 금지하고 있다. 여기서 ‘부정한 수단, 계획 또는 기교’란 사회통념상 부정하다고 인정되는 일체의 수단, 계획 또는 기교를 말한다. 나아가 어떠한 행위를 부정하다고 할지는 그 행위가 법령 등에서 금지된 것인지, 다른 투자자들로 하여금 잘못된 판단을 하게 함으로써 공정한 경쟁을 해치고 선의의 투자자에게 손해를 전가하여 자본시장의 공정성, 신뢰성 및 효율성을 해칠 위험이 있는지를 고려하여 판단하여야 한다(대법원 2014. 1. 16. 선고 2013도9933 판결 등 참조).\n한편 자본시장법 제178조

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

dataset_dict:  {'prompt': '\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[손해배상(기)]〈혼인파탄후 제3자와의 성적행위로 인한 손해배상청구 사건〉 [공2014하,2361]\n판시사항:[1] 제3자가 부부의 일방과 부정행위를 함으로써 부부공동생활을 침해하거나 유지를 방해하고 그에 대한 배우자로서 권리를 침해하여 배우자에게 정신적 고통을 가하는 행위가 불법행위를 구성하는지 여부(원칙적 적극)\n[2] 부부가 아직 이혼하지 아니하였지만 실질적으로 부부공동생활이 파탄되어 회복할 수 없을 정도의 상태에 이른 경우, 제3자가 부부의 일방과 한 성적인 행위가 배우자에 대하여 불법행위를 구성하는지 여부(소극) / 이러한 법률관계는 재판상 이혼청구가 계속 중에 있다거나 재판상 이혼이 청구되지 않은 상태라고 하더라도 마찬가지인지 여부(적극)\n\n        assistant : \n    ', 'label': '\n상고이유를 판단한다.\n1. 부부는 동거하며 서로 부양하고 협조하여야 하는 의무를 진다(민법 제826조). 부부는 정신적·육체적·경제적으로 결합된 공동체로서 서로 협조하고 보호하여 부부공동생활로서의 혼인이 유지되도록 상호 간에 포괄적으로 협력할 의무를 부담하고 그에 관한 권리를 가진다. 이러한 동거의무 내지 부부공동생활 유지의무의 내용으로서 부부는 부정행위를 하지 아니하여야 하는 성적(성적) 성실의무를 부담한다. 이에 따라 부부의 일방이 부정행위를 한 경우에 이는 민법 제840조에 따라 재판상 이혼사유가 되고, 부부의 일방은 그로 인하여 배우자가 입게 된 정신적 고통에 대하여 불법행위에 의한 손해배상의무를 진다.\n한편 제3자도 타인의 부부공동생활에 개입하여 그 부부공동생활의 파탄을 초래하는 등 그 혼인의 본질에 해당하는 부부공동생활을 방해하여서는 아니 된다. 제3자가 부

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [8]:
## ---  LoRA --- ##
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    # modules_to_save=["q_proj", "k_proj", "v_proj"],
)
model = get_peft_model(base_model, config)
model.print_trainable_parameters()

peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

trainable params: 4,718,592 || all params: 8,172,867,584 || trainable%: 0.0577


In [9]:
raise ValueError()

ValueError: 

In [10]:
num_epochs = 30
learning_rate = 2e-5
optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
)
lr_scheduler = StepLR(optimizer, step_size=1, gamma=0.85)

In [11]:
# accelerator.prepare(
#     train_dataloader, valid_dataloader, model, optimizer, lr_scheduler
# )

In [12]:
def save_to_json(output_filename, epoch, train_epoch_loss, train_epoch_ppl, val_epoch_loss, val_epoch_ppl):
    metrics_data = {
        "epoch" : epoch,
        "train_epoch_loss": train_epoch_loss,
        "train_epoch_perplexity": train_epoch_ppl,
        "val_epoch_loss": val_epoch_loss,
        "val_epoch_perplexity": val_epoch_ppl
    }
    with open(output_filename, "a+") as f:
        json.dump(metrics_data, f)
        f.write('\n')


In [13]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [14]:
from tqdm import tqdm

device = "cuda"
model = model.to(device)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch)
        with torch.autocast(device_type='cuda'):
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.detach().float()
        
        # accelerator.backward(loss)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
    

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")
    save_to_json("./finetuning_train_result.json", epoch, float(train_epoch_loss), float(train_ppl.detach().cpu()), float(eval_epoch_loss), float(eval_ppl.detach().cpu()))
    

  0%|          | 0/701 [00:00<?, ?it/s]

100%|██████████| 88/88 [00:26<00:00,  3.29it/s]


epoch=0: train_ppl=tensor(22.6323, device='cuda:0') train_epoch_loss=tensor(3.1194, device='cuda:0') eval_ppl=tensor(13.7109, device='cuda:0') eval_epoch_loss=tensor(2.6182, device='cuda:0')


100%|██████████| 88/88 [00:26<00:00,  3.29it/s]


epoch=1: train_ppl=tensor(12.7047, device='cuda:0') train_epoch_loss=tensor(2.5420, device='cuda:0') eval_ppl=tensor(11.6100, device='cuda:0') eval_epoch_loss=tensor(2.4519, device='cuda:0')


...

epoch=27: train_ppl=tensor(4.9387, device='cuda:0') train_epoch_loss=tensor(1.5971, device='cuda:0') eval_ppl=tensor(11.0040, device='cuda:0') eval_epoch_loss=tensor(2.3983, device='cuda:0')


100%|██████████| 88/88 [00:26<00:00,  3.29it/s]


epoch=28: train_ppl=tensor(4.8220, device='cuda:0') train_epoch_loss=tensor(1.5732, device='cuda:0') eval_ppl=tensor(11.2609, device='cuda:0') eval_epoch_loss=tensor(2.4213, device='cuda:0')


100%|██████████| 88/88 [00:26<00:00,  3.29it/s]

epoch=29: train_ppl=tensor(4.7057, device='cuda:0') train_epoch_loss=tensor(1.5488, device='cuda:0') eval_ppl=tensor(11.5949, device='cuda:0') eval_epoch_loss=tensor(2.4506, device='cuda:0')


In [15]:
model.save_pretrained(f"./{peft_model_id}/")

/home/eahc00/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
from peft import PeftModel

text_decoder_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B"
peft_model_id = f"MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# LoRA 적용 모델 불러오기
base_model = AutoModelForCausalLM.from_pretrained(
    text_decoder_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True, 
    quantization_config=bnb_config
    )


load_model = PeftModel.from_pretrained(base_model, peft_model_id)
loaded_state_dict = load_file("/home/eahc00/NLP/term_project/MLP-KTLim/llama-3-Korean-Bllossom-8B_Lora/adapter_model.safetensors")
load_model.load_state_dict(loaded_state_dict, strict=False)
tokenizer = AutoTokenizer.from_pretrained(text_decoder_id)

ds = get_dataset(split="test", get_prompt_only=True)

load_model.to('cuda')

i = 1
inputs = tokenizer(ds[i]['prompt'], return_tensors="pt")
print(ds[i]['prompt'])
print(ds[i]['label'])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.
        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘
        
 title:[배임]〈대물변제예약 사안에서 배임죄 사건〉 [공2014하,1923]
판시사항:[1] 채권 담보 목적으로 부동산에 관한 대물변제예약을 체결한 채무자가 대물로 변제하기로 한 부동산을 제3자에게 처분한 경우, 배임죄가 성립하는지 여부(소극)
[2] 채무자인 피고인이 채권자 갑에게 차용금을 변제하지 못할 경우 자신의 어머니 소유 부동산에 대한 유증상속분을 대물변제하기로 약정한 후 유증을 원인으로 위 부동산에 관한 소유권이전등기를 마쳤음에도 이를 제3자에게 매도함으로써 갑에게 손해를 입혔다고 하여 배임으로 기소된 사안에서, 피고인이 ‘타인의 사무를 처리하는 자’의 지위에 있다고 볼 수 없는데도, 이와 다른 전제에서 유죄를 인정한 원심판결에 법리오해의 위법이 있다고 한 사례

        assistant : 
    

1. 배임죄는 타인의 사무를 처리하는 자가 그 임무에 위배하는 행위로 재산상 이익을 취득하여 사무의 주체인 타인에게 손해를 가함으로써 성립하는 것이므로 그 범죄의 주체는 타인의 사무를 처리하는 지위에 있어야 한다. 여기에서 ‘타인의 사무’를 처리한다고 하려면 당사자 관계의 본질적 내용이 단순한 채권채무 관계를 넘어서 그들 간의 신임관계에 기초하여 타인의 재산을 보호 또는 관리하는 데 있어야 하고, 그 사무가 타인의 사무가 아니고 자기의 사무라면 그 사무의 처리가 타인에게 이익이 되어 타인에 대하여 이를 처리할 의무를 부담하는 경우라도 그는 타인의 사무를 처리하는 자에 해당하지 아니한다(대법원 2011. 1. 20. 선고 2008도10479 전원합의체 판결 참조).
채무자가 채권자에 대하여 소비대차 등으로 인한 채무를 부담하고 이를 담보하기 위하여 장래에 부동산의 소유권을 이전하기로 하는 내용의 대물변제예약에서, 그 약정의 내용에 좇은 이행을 

In [18]:
device = "cuda"

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = load_model.generate(input_ids=inputs["input_ids"], max_new_tokens=512)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:144783 for open-end generation.


['\n        system : 한국말로만 대답하고 최대한 간결하고 알기쉽게 정리해줘.\n        user : 사건의 title과 판시사항을 보고 판결 결과와 그 이유를 예측해줘\n        \n title:[배임]〈대물변제예약 사안에서 배임죄 사건〉 [공2014하,1923]\n판시사항:[1] 채권 담보 목적으로 부동산에 관한 대물변제예약을 체결한 채무자가 대물로 변제하기로 한 부동산을 제3자에게 처분한 경우, 배임죄가 성립하는지 여부(소극)\n[2] 채무자인 피고인이 채권자 갑에게 차용금을 변제하지 못할 경우 자신의 어머니 소유 부동산에 대한 유증상속분을 대물변제하기로 약정한 후 유증을 원인으로 위 부동산에 관한 소유권이전등기를 마쳤음에도 이를 제3자에게 매도함으로써 갑에게 손해를 입혔다고 하여 배임으로 기소된 사안에서, 피고인이 ‘타인의 사무를 처리하는 자’의 지위에 있다고 볼 수 없는데도, 이와 다른 전제에서 유죄를 인정한 원심판결에 법리오해의 위법이 있다고 한 사례\n\n        assistant : \n    \n상고이유를 판단한다.\n1. 원심판결 이유를 원심이 채택한 증거에 비추어 살펴보면, 다음과 같은 사실을 알 수 있다.\n가. 피고인은 1988. 6. 1. 피고인과 그의 부인인 공소외인 공소외인(이하 ‘공소외인’이라 한다) 사이의 배우자간 재산공정한혼이라는 재산공정한혼에 관한 혼인유치의 증명으로 ‘공소외인 소유 (위 1) 부동산에 대한 유증증서, (위 1) 부동산의 공시가액에 대한 유증증서’를 변제예금으로 지정한 채무를 이행할 경우, 위 (위 1) 부동산을 공소외인에게 모두 처분한다고 약정한 뒤 2006. 1. 6. 위 (위 1) 부동산에 관한 대물변제계약서를 공소외인으로 작성하여 공소외인이 중도정산한 금액을 지급받고 2006. 2. 11. 공소외인에게 등기를 하였다(이하 위 (위 1) 부동산을 취득하는 채무자로서의 피고인을 ‘피고인’이라 하고, 위 (위 1) 부동산을 취득하는 공소외인을 ‘피해자’라 한다. 이하 위와 같은